In [1]:
from molSimplify.Informatics.autocorrelation import*
from ase import io, Atoms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.kernel_ridge import KernelRidge

from sklearn.model_selection import LeaveOneOut

In [2]:

rel_logs = pickle.load(open("/home/michael/PycharmProjects/gaussian_job_manage/complete_logs.p", "r"))

valid_cats = pickle.load(open("../molsimp_catalysts/well_behaved_cats.p", "rb"))

rel_coms = [x.replace(".log", ".com") for x in rel_logs if "VAC" in x and x.split("/")[4] in valid_cats]

cat_data = pickle.load(open("/home/michael/PycharmProjects/gaussian_job_manage/catalyst_data.p", "r"))
oh_values = []
cat_names = []
for comfilllll in rel_coms:
    if comfilllll.split("/")[4] in valid_cats:
        oh_values.append(cat_data[comfilllll.split("/")[4]][0])
        cat_names.append(comfilllll.split("/")[4])
print(len(oh_values))

216


In [3]:
def make_rac(comfile, depth, is_oct):
    print(comfile)
    with open(comfile, "r") as f:
        lines = f.readlines()
    
    thing = io.read(comfile, format='gaussian')

    
    properties = ['electronegativity', 'nuclear_charge', 'topology', 'size', 'effective_nuclear_charge']
    
    #thing = Atoms(elements, positions=xyz)
    thing.write("tmp_.xyz")
    #print(os.popen("head -n3 tmp_.xyz").readlines()[2])
    this_mol = mol3D() # mol3D instance
    this_mol.readfromxyz("tmp_.xyz")
    full_autocorrs = np.asarray(generate_atomonly_autocorrelations(this_mol, 0, True, depth=depth, oct=is_oct)['results']).flatten()
    delta_metric = np.asarray(generate_atomonly_deltametrics(this_mol, 0, True, depth=depth, oct=is_oct)['results']).flatten()
    #print(delta_metric)

    if is_oct:
        num_connectors = 6
    else:
        num_connectors = 5
    distances = []
    origin = thing.get_positions()[0]
    for xyz in thing.get_positions():
        distances.append(np.sqrt((xyz[0]-origin[0])**2+(xyz[1]-origin[1])**2+(xyz[2]-origin[2])**2))

    nearest_neighbours = np.argpartition(distances, num_connectors)
    nn = [x for x in nearest_neighbours[:num_connectors+1] if x != 0]
    rest_of_autoz = np.zeros(shape=(len(properties), depth+1))
    rest_of_deltas = np.zeros(shape=(len(properties), depth))
    for idx, p in enumerate(properties):
        rest_of_autoz[idx] = atom_only_autocorrelation(this_mol, p, depth, nn)
        rest_of_deltas[idx] = atom_only_deltametric(this_mol, p, depth, nn)[1:]
        
    rest_delta = rest_of_deltas.flatten()
    rest = rest_of_autoz.flatten()
    return np.concatenate((full_autocorrs, delta_metric, rest, rest_delta), axis=None)


In [4]:
racs = np.zeros(shape=(len(rel_coms), 75))

for idx, com in enumerate(rel_coms):
    if "31a" or "3a1" or "32a" or "41a" in com:
        is_oct = False
    else:
        is_oct = True
    racs[idx] = make_rac(com, 3, is_oct)

/home/michael/ht_scaling_results/Fe_31a_17/VAC/HS/Fe_31a_17_VAC_HS.com
/home/michael/ht_scaling_results/Co_31a_23/VAC/HS/Co_31a_23_VAC_HS.com
/home/michael/ht_scaling_results/Cr_32a_35/VAC/HS/back/Cr_32a_35_VAC_HS-b_3e-1.com
/home/michael/ht_scaling_results/Mn_32a_35/VAC/HS/Mn_32a_35_VAC_HS.com
/home/michael/ht_scaling_results/Ru_32a_34/VAC/LS/Ru_32a_34_VAC_LS.com
/home/michael/ht_scaling_results/Mn_31a_23/VAC/HS/Mn_31a_23_VAC_HS.com
/home/michael/ht_scaling_results/Fe_3a1_5_/VAC/HS/Fe_3a1_5_VAC_HS.com
/home/michael/ht_scaling_results/Ni_3a1_5_/VAC/HS/Ni_3a1_5_VAC_HS.com
/home/michael/ht_scaling_results/Cr_3a1_28/VAC/HS/Cr_3a1_28_VAC_HS.com
/home/michael/ht_scaling_results/Co_3a1_17/VAC/HS/Co_3a1_17_VAC_HS.com
/home/michael/ht_scaling_results/Ru_3a1_1_/VAC/LS/Ru_3a1_1_VAC_LS.com
/home/michael/ht_scaling_results/Fe_3a1_22/VAC/HS/Fe_3a1_22_VAC_HS.com
/home/michael/ht_scaling_results/Co_penta_4/VAC/HS/Co_penta_4_VAC_HS.com
/home/michael/ht_scaling_results/Ni_3a1_24/VAC/HS/Ni_3a1_24_VAC_HS

/home/michael/ht_scaling_results/Fe_31a_23/VAC/HS/Fe_31a_23_VAC_HS.com
/home/michael/ht_scaling_results/Ni_3a1_10/VAC/HS/Ni_3a1_10_VAC_HS.com
/home/michael/ht_scaling_results/Fe_3a1_19/VAC/HS/Fe_3a1_19_VAC_HS.com
/home/michael/ht_scaling_results/Co_3a1_5_/VAC/HS/Co_3a1_5_VAC_HS.com
/home/michael/ht_scaling_results/Co_31a_19/VAC/HS/Co_31a_19_VAC_HS.com
/home/michael/ht_scaling_results/Mn_32a_34/VAC/HS/Mn_32a_34_VAC_HSb.com
/home/michael/ht_scaling_results/Fe_penta_5/VAC/HS/Fe_penta_5_VAC_HS.com
/home/michael/ht_scaling_results/Ni_32a_35/VAC/HS/Ni_32a_35_VAC_HS.com
/home/michael/ht_scaling_results/Cr_3a1_19/VAC/HS/Cr_3a1_19_VAC_HS.com
/home/michael/ht_scaling_results/Ru_31a_11/VAC/LS/Ru_31a_11_VAC_LS.com
/home/michael/ht_scaling_results/Cr_penta_2/VAC/HS/Cr_penta_2_VAC-HS.com
/home/michael/ht_scaling_results/Cr_penta_4/VAC/HS/Cr_penta_4_VAC_HS.com
/home/michael/ht_scaling_results/Cr_41a_3_/VAC/HS/Cr_41a_3_VAC_HS-cartesian.com
/home/michael/ht_scaling_results/Ru_32a_35/VAC/LS/Ru_32a_35_VA

In [5]:
from sklearn.metrics import mean_squared_error
from math import sqrt

alphas = [0.001, 0.01, 0.1, 0.0001]
const_bounds = [(1e-3, 1e3), (1e-2, 1e2), (0.1, 10)]
rbf_bound = [(1e-3, 1e3), (1e-2, 1e2), (0.1, 10)]
rbf_scale = [0.1, 1, 10]
const_scale = [0.1, 1, 10]


for al in alphas:
    for const_b in const_bounds:
        for rbf_b in rbf_bound:
            for rbf_s in rbf_scale:
                for const_c in const_scale:
                    stdzz = []
                    preds = []
                    test_ohs = []
                    errs = []

                    loo = LeaveOneOut()

                    for train_index, test_index in loo.split(racs):
                        X_train, X_test = racs[train_index], racs[test_index]
                        y_train, y_test = np.asarray(oh_values)[train_index], np.asarray(oh_values)[test_index]

                        #kernel =  ConstantKernel(constant_value_bounds=(1e-2, 1e2))*RBF(.1, length_scale_bounds=(1e-3, 1e3))

                        kernel = ConstantKernel(const_c, const_b) * RBF(rbf_s, rbf_b)
                        gp = GaussianProcessRegressor(kernel=kernel, alpha=al, n_restarts_optimizer=20)
                        #gp = GaussianProcessRegressor()
                        gp.fit(X_train, y_train)

                        pred, std = gp.predict(X_test, y_test)
                        stdzz.append(std[0])
                        preds.append(pred[0])
                        errs.append(abs(y_test[0]-pred[0]))
                        test_ohs.append(y_test[0])
                        #print(np.asarray(cat_names)[test_index][0])
                        #print("Uncertainty is: {:.3f}".format(std[0]))
                        #print("Prediction is: {:.3f}".format(pred[0]))
                        #print("True value is: {:.3f}".format(y_test[0]))
                        #print("Err is: {:.3f}".format(abs(y_test[0]-pred[0])))
                    print("alpha in gpr: {}".format(al))
                    print("Const kernel bounds: {}".format(const_b))
                    print("RBF kernel bounds: {}".format(rbf_b))
                    print("RBF kernel scale: {}".format(rbf_s))
                    print("Const kernel scale: {}".format(const_c))
                    print("RMSE: {:.3f}".format(sqrt(mean_squared_error(np.asarray(test_ohs), np.asarray(preds)))))

/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 1.00515549e-04, -3.24693814e-23]), 'nit': 3, 'funcalls': 47}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([4.35051558e-05, 5.95042828e-19]), 'nit': 7, 'funcalls': 50}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([3.90310658e-05, 1.39791660e-21]), 'nit': 7, 'funcalls': 49}
  ConvergenceWarni

alpha in gpr: 0.001
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.001, 1000.0)
RBF kernel scale: 0.1
Const kernel scale: 0.1
RMSE: 0.274


/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 3.68843556e-05, -1.05270362e-07]), 'nit': 6, 'funcalls': 49}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([5.97723276e-05, 2.30811200e-17]), 'nit': 7, 'funcalls': 55}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 1.63839050e-05, -2.16376229e-05]), 'nit': 11, 'funcalls': 58}
  ConvergenceWa

alpha in gpr: 0.001
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.001, 1000.0)
RBF kernel scale: 0.1
Const kernel scale: 1
RMSE: 0.274


/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([1.32143745e-05, 2.84840044e-18]), 'nit': 2, 'funcalls': 45}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([4.89597059e-05, 1.01025711e-18]), 'nit': 5, 'funcalls': 47}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([4.66146147e-06, 1.68067910e-05]), 'nit': 15, 'funcalls': 61}
  ConvergenceWarnin

...

/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-2.83124179e-06,  1.12309134e-05]), 'nit': 9, 'funcalls': 52}
  ConvergenceWarning)


alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.001, 1000.0)
RBF kernel scale: 10
Const kernel scale: 0.1
RMSE: 0.238


/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([9.50383594e-07, 1.43645974e-05]), 'nit': 9, 'funcalls': 55}
  ConvergenceWarning)
/home/michael/anaconda3/envs/molSimplify/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 3.86632529e-06, -1.24394130e-05]), 'nit': 13, 'funcalls': 57}
  ConvergenceWarning)


alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.001, 1000.0)
RBF kernel scale: 10
Const kernel scale: 1
RMSE: 0.238
alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.001, 1000.0)
RBF kernel scale: 10
Const kernel scale: 10
RMSE: 0.238
alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.01, 100.0)
RBF kernel scale: 0.1
Const kernel scale: 0.1
RMSE: 0.279
alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.01, 100.0)
RBF kernel scale: 0.1
Const kernel scale: 1
RMSE: 0.279
alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.01, 100.0)
RBF kernel scale: 0.1
Const kernel scale: 10
RMSE: 0.279
alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.01, 100.0)
RBF kernel scale: 1
Const kernel scale: 0.1
RMSE: 0.279
alpha in gpr: 0.01
Const kernel bounds: (0.001, 1000.0)
RBF kernel bounds: (0.01, 100.0)
RBF kernel scale: 1
Const kernel sc

KeyboardInterrupt: 